# Centrummaten, Spreidingsmaten en Samenhang

In dit labo bestuderen we hoe we kansverdelingen kunnen karakteriseren met behulp van samenvattende statistieken. We berekenen verwachtingswaarden, varianties, covarianties en correlaties.

**Leerdoelen:**
- Verwachtingswaarde (gemiddelde) berekenen en interpreteren
- Variantie en standaardafwijking als spreidingsmaten gebruiken
- Mediaan, modus en percentielen bepalen
- Covariantie en correlatie berekenen voor lineaire samenhang
- Het verschil begrijpen tussen ongecorreleerd en onafhankelijk

In [ ]:
import numpy as np
import plotly.express as px
from scipy import stats

# Set random seed
rng = np.random.default_rng(42)

## Centrummaten

Centrummaten geven het "zwaartepunt" van een kansverdeling aan.

### Verwachtingswaarde (Expected Value)

De **verwachtingswaarde** $\mathbb{E}[x]$ of $\mu$ is het gemiddelde dat we verwachten bij oneindig veel observaties.

- Discreet: $\mathbb{E}[x] = \sum_x x \cdot P(x)$
- Continu: $\mathbb{E}[x] = \int x \cdot p(x) dx$

## ✍️
Bereken de verwachtingswaarde voor een eerlijke dobbelsteen.

## ✍️
Simuleer dobbelsteenworp en toon aan dat het steekproefgemiddelde convergeert naar de verwachtingswaarde (Wet van Grote Aantallen).

### Lineaire transformaties van verwachtingswaarden

De verwachtingswaarde heeft handige eigenschappen bij lineaire transformaties:

$$\mathbb{E}[ax + b] = a \cdot \mathbb{E}[x] + b$$
$$\mathbb{E}[x + y] = \mathbb{E}[x] + \mathbb{E}[y]$$

## ✍️
Verifieer deze eigenschappen numeriek.

### Mediaan en Modus

- **Mediaan**: De waarde waarbij 50% van de kansmassa links en rechts ligt
- **Modus**: De meest waarschijnlijke waarde (hoogste kans bij discreet, maximum van PDF bij continu)

## ✍️
Vergelijk verwachtingswaarde, mediaan en modus voor verschillende verdelingen.

## Spreidingsmaten

Spreidingsmaten geven aan hoe sterk waarden variëren rond het centrum.

### Variantie en Standaardafwijking

**Variantie**: $\text{Var}(x) = \mathbb{E}[(x - \mu)^2] = \mathbb{E}[x^2] - (\mathbb{E}[x])^2$

**Standaardafwijking**: $\sigma = \sqrt{\text{Var}(x)}$

## ✍️
Bereken de variantie en standaardafwijking voor een eerlijke dobbelsteen.

### Eigenschappen van variantie

$$\text{Var}(ax + b) = a^2 \cdot \text{Var}(x)$$

Voor onafhankelijke variabelen:
$$\text{Var}(x + y) = \text{Var}(x) + \text{Var}(y)$$

## ✍️
Verifieer deze eigenschappen.

### Vergelijking van spreiding tussen verdelingen

## ✍️
Visualiseer normale verdelingen met verschillende standaardafwijkingen maar hetzelfde gemiddelde.

### Interkwartielafstand (IQR)

De **IQR** is een robuuste maat voor spreiding, minder gevoelig voor uitschieters:

$$\text{IQR} = Q_3 - Q_1 = P_{75} - P_{25}$$

Uitschieters worden vaak gedefinieerd als waarden buiten $[Q_1 - 1.5 \times \text{IQR}, Q_3 + 1.5 \times \text{IQR}]$

## ✍️
Bereken IQR en detecteer uitschieters in een dataset met extreme waarden.

## Covariantie en Correlatie

Voor twee random variabelen meten we de **lineaire samenhang**.

### Covariantie

$$\text{Cov}(x, y) = \mathbb{E}[(x - \mathbb{E}[x])(y - \mathbb{E}[y])]$$

- Positief: gelijktijdige afwijkingen hebben dezelfde richting
- Negatief: gelijktijdige afwijkingen hebben tegengestelde richting  
- ~0: geen lineaire samenhang

## ✍️
Bereken covariantie voor verschillende relaties tussen variabelen.

### Correlatie (Pearson)

**Correlatiecoëfficiënt**: Genormaliseerde covariantie

$$\rho_{xy} = \frac{\text{Cov}(x, y)}{\sigma_x \sigma_y}$$

Eigenschappen:
- $-1 \leq \rho \leq 1$
- $|\rho| = 1$: perfecte lineaire relatie
- $\rho = 0$: geen lineaire correlatie
- Schaal-onafhankelijk (in tegenstelling tot covariantie)

## ✍️
Bereken correlaties en vergelijk met covarianties.

### Correlatie bij verschillende sterktes

## ✍️
Visualiseer relaties met verschillende correlatiesterktes.

### Covariantie ≈ 0 betekent NIET onafhankelijkheid!

Covariantie/correlatie meet alleen **lineaire** samenhang. Er kan een sterk niet-lineair verband bestaan met correlatie ≈ 0.

## ✍️
Demonstreer niet-lineaire afhankelijkheid met correlatie ≈ 0.

### Covariantiematrix

Voor een vector van random variabelen kunnen we alle covarianties in een matrix samenvatten:

$$\pmb{\Sigma}_{ij} = \text{Cov}(x_i, x_j)$$

- Diagonaal: varianties
- Off-diagonal: covarianties

## ✍️
Bereken en visualiseer een covariantiematrix voor meerdere variabelen.